In [12]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim import models

/Users/mmiyazaki/dataiku/Design/DATA_DIR/code-envs/python/japan-nlp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Read recipe inputs
ramen_clusters_named = dataiku.Dataset("ramen_clusters_named")
df = ramen_clusters_named.get_dataframe()

In [3]:
list_vocabs = df[df['cluster_labels'].isin(['接客などのサービス', '具材・素材・味'])]['words_concat'].values

In [4]:
ramen_word = list_vocabs[0].split(",") + list_vocabs[1].split(",")

In [7]:
folder_path = dataiku.Folder("aLTWBozg").get_path()
text_path = folder_path + "/ramen_corpus.txt"

In [9]:
f = open(text_path,'r',encoding="utf-8")
trainings = []

In [10]:
for i,data in enumerate(f):
    word = data.replace("'",'').replace('[','').replace(']','').replace(' ','').replace('\n','').split(",")
    trainings.append([i for i in word if i in ramen_word])

In [31]:
len(trainings)

211

In [13]:
dictionary = corpora.Dictionary(trainings)

In [50]:
len(dictionary)

2

In [15]:
corpus = list(map(dictionary.doc2bow, trainings))

In [16]:
test_model = models.TfidfModel(corpus)

In [17]:
corpus_tfidf = test_model[corpus]

In [19]:
# id->単語へ変換
texts_tfidf = [] # id -> 単語表示に変えた文書ごとのTF-IDF
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary[word[0]],word[1]])
    texts_tfidf.append(text_tfidf)

In [28]:
from operator import itemgetter

texts_tfidf_sorted_top20 = [] 

#TF-IDF値を高い順に並び替え上位単語20個に絞る。
for i in range(len(texts_tfidf)):
    soted = sorted(texts_tfidf[i], key=itemgetter(1),reverse=True)
    soted_top20 = soted[:20]
    word_list = []
    for k in range(len(soted_top20)):
        word = soted_top20[k][0]
        word_list.append(word)
    texts_tfidf_sorted_top20.append(word_list)

In [37]:
raw_ramen_df = dataiku.Dataset("raw_ramen").get_dataframe()

In [38]:
raw_ramen_df.head()

,store_id,store_name,address,score,ward,review_cnt,review,review_star
0,1,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,3.98,東京都内,479,2020年、ラーメン103杯目(^^)本日、朝イチで営業を済ませ、次の打ち合わせまで時間が空...,4.8
1,1,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,3.98,東京都内,479,ミシュラン東京2019 ビルグルマン受賞店の「手打式超多加水麺 ののくら」さんに伺いました。...,3.8
2,1,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,3.98,東京都内,479,平日の13時過ぎに伺いました＼(^o^)／駅から少し歩いた通り沿いにあり、外観はシンプル落ち...,3.8
3,1,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,3.98,東京都内,479,食べログ100名店簡単にお店の場所:亀有駅に着いて、両さんがお出迎え、南口から徒歩5分くらい...,3.8
4,1,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,3.98,東京都内,479,エグい行列が嫌でしばらく行ってませんでしたが、土曜日の「記帳予約制」なる制度が始まったので久...,4.5


In [44]:
ramen_by_restaurant = raw_ramen_df.groupby(['store_name', 'address', 'ward', 'score', 'review_cnt'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)

In [48]:
ramen_by_restaurant.shape

(211, 6)

In [47]:
ramen_by_restaurant.head()

,store_name,address,ward,score,review_cnt,review
112,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,東京都内,3.98,479,2020年、ラーメン103杯目(^^)本日、朝イチで営業を済ませ、次の打ち合わせまで時間が空...
0,Homemade Ramen 麦苗,東京都品川区南大井6-11-10,東京都内,3.97,731,2020/08/03(月)5度目の訪問！現在は蜜を避けるために、事前記帳(予約)式になってい...
191,麺尊 RAGE,東京都杉並区松庵3-37-22レンツェン松庵 1F,東京都内,3.96,723,5年連続で「ミシュランガイド東京」のビブグルマンに掲載され、「The Tabelog Awa...
110,宍道湖しじみ中華蕎麦 琥珀,東京都大田区西六郷2-1-3,東京都内,3.95,296,本日は貴重な平日のお休みだった為、普段は中々足を運ぶ事の出来ない蒲田方面へ。11時40分に蒲...
25,らぁ麺や 嶋,東京都渋谷区本町3-41-12,東京都内,3.95,177,今年、西新宿五丁目に新しいラーメン屋が出来た。まだ1年も経っていないだろうにすでに期待度が非...


In [0]:
ramen_by_restaurant['texts_tfidf_sorted_top20'] = texts_tfidf_sorted_top20